In [ ]:
import numpy as np
from PIL import Image
from glob import glob
import cv2
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from skimage.io import imread
from skimage.color import rgb2gray
import os
from scipy.ndimage import center_of_mass
import shutil
import pandas as pd
from tqdm import tqdm

In [ ]:

def calculate_extended_line(start_point, end_point, width, height):
    """
    주어진 두 점을 잇는 직선을 이미지 경계까지 확장합니다.

    Args:
        start_point (tuple): 시작점 (x1, y1).
        end_point (tuple): 끝점 (x2, y2).
        width (int): 이미지의 가로 크기.
        height (int): 이미지의 세로 크기.

    Returns:
        tuple: 직선의 시작점과 끝점 ((x1, y1), (x2, y2)).
    """
    x1, y1 = start_point
    x2, y2 = end_point

    # 기울기와 y절편 계산
    if x2 != x1:
        slope = (y2 - y1) / (x2 - x1)
        intercept = y1 - slope * x1
    else:
        slope = None  # 수직선
        intercept = None

    # 경계와 교차점 계산
    points = []
    if slope is not None:
        # x=0 (왼쪽 경계)
        y = slope * 0 + intercept
        if 0 <= y < height:
            points.append((0, y))
        # x=width (오른쪽 경계)
        y = slope * width + intercept
        if 0 <= y < height:
            points.append((width, y))
        # y=0 (상단 경계)
        x = (0 - intercept) / slope if slope != 0 else np.inf
        if 0 <= x < width:
            points.append((x, 0))
        # y=height (하단 경계)
        x = (height - intercept) / slope if slope != 0 else np.inf
        if 0 <= x < width:
            points.append((x, height))
    else:
        # 수직선의 경우
        if 0 <= x1 < width:
            points.append((x1, 0))
            points.append((x1, height))

    # 유효한 교차점이 두 개 이상이면 반환
    if len(points) >= 2:
        return points[0], points[1]
    else:
        raise ValueError("No valid extended line points within image boundaries.")

# Load the binary image (replace 'your_binary_image.png' with your actual image file)
def calculate_line_points(slope, intercept, width, height):
    # 이미지 경계선(좌우, 상하)에 대해 교차점 계산
    points = []
    # x=0 (왼쪽 경계)
    y = slope * 0 + intercept
    if 0 <= y < height:
        points.append((0, y))
    # x=width (오른쪽 경계)
    y = slope * width + intercept
    if 0 <= y < height:
        points.append((width, y))
    # y=0 (상단 경계)
    x = (0 - intercept) / slope if slope != 0 else np.inf
    if 0 <= x < width:
        points.append((x, 0))
    # y=height (하단 경계)
    x = (height - intercept) / slope if slope != 0 else np.inf
    if 0 <= x < width:
        points.append((x, height))
    return points
def find_max_x_on_line(points, slope, intercept, tolerance=1):
    """
    주어진 점들 중 직선 방정식에 부합하며 x 값이 가장 큰 좌표를 찾습니다.
    
    Args:
        points (list of tuples): (x, y) 형식의 점들.
        slope (float): 직선의 기울기.
        intercept (float): 직선의 y-절편.
        tolerance (float): y = mx + b 오차 허용 범위.
    
    Returns:
        tuple: 직선 위에 위치하며 x 값이 가장 큰 점 (x, y).
    """
    max_x_point =0
    max_x_value = float('-inf')

    for x, y in points:
        # 직선 방정식 y = mx + b 확인
        expected_y = slope * x + intercept
        if abs(y - expected_y) <= tolerance:  # y 값이 직선 방정식에 부합
        
            if x > max_x_value:  # x 값이 현재 최대보다 크면 갱신
                max_x_value = x
                max_x_point = (x, y)

    return max_x_point

def calculate_angle_between_lines(start1, end1, start2, end2):
    """
    두 직선의 방향 벡터를 사용하여 사잇각을 계산합니다.
    
    Args:
        start1, end1: 첫 번째 직선의 시작점과 끝점 (x, y).
        start2, end2: 두 번째 직선의 시작점과 끝점 (x, y).
    
    Returns:
        angle: 두 직선 사이의 각도 (degrees).
    """
    # 첫 번째 직선의 방향 벡터
    vector1 = np.array([end1[0] - start1[0], end1[1] - start1[1]])
    # 두 번째 직선의 방향 벡터
    vector2 = np.array([end2[0] - start2[0], end2[1] - start2[1]])
    
    # 벡터 내적
    dot_product = np.dot(vector1, vector2)
    # 벡터 크기 계산
    norm1 = np.linalg.norm(vector1)
    norm2 = np.linalg.norm(vector2)
    
    # 코사인 값 계산
    cos_theta = dot_product / (norm1 * norm2)
    # 각도 계산 (라디안 -> 도)
    angle = np.degrees(np.arccos(np.clip(cos_theta, -1.0, 1.0)))
    return angle


ddh_grade=[0,0,0]
total_df=pd.DataFrame(columns=['case','image','alpha','beta','DDH','DDH_predict'])
total_count=0
test_df=pd.read_csv('../../data/Graf_algorithm/dataset/test_data.csv')
mask_list=[]
image_list=[]
for i in range(len(test_df)):
    mask_list.append('../../data/Graf_algorithm/test/PAN/'+str(test_df['case'][i])+'/'+os.path.splitext(test_df['image'][i])[0])
    image_list.append('../../data/Graf_algorithm/dataset/'+str(test_df['case'][i])+'/image/'+test_df['image'][i])
for i in tqdm(range(len(mask_list))):
    # base line and bony roof points
    try:
        mask_path = mask_list[i]+'/2.png'
        image_path=image_list[i]
        binary_image = np.array(Image.open(mask_path))
        raw_image = np.array(Image.open(image_path))
        rows, cols = np.where(binary_image > 0.5)
        points = np.column_stack((cols, rows))  
        pca = PCA(n_components=2)
        pca.fit(points) 
        center = pca.mean_  # 중심점 (평균 좌표)
        components = pca.components_  # 주성분 벡터
        slope = components[0][1] / components[0][0]  # 기울기
        intercept = center[1] - slope * center[0]  # y절편
        bony_roof_points = find_max_x_on_line(points, slope, intercept)
        # 5. 이미지 경계와 교차점 계산
        height, width = binary_image.shape
        line_points = calculate_line_points(slope, intercept, width, height)
        # 직선의 시작점과 끝점 설정
        if len(line_points) >= 2:
            base_line_start, base_line_end = line_points[:2]
        else:
            raise ValueError("No valid line points within image boundaries")

        #Center of labrum
        mask_path = mask_list[i]+'/1.png'
        binary_image = np.array(Image.open(mask_path))
        center_labrum_points = center_of_mass(binary_image)
        center_labrum_points=(int(center_labrum_points[1]),int(center_labrum_points[0]))

        #Bony rim point
        mask_path = mask_list[i]+'/3.png'
        binary_image = np.array(Image.open(mask_path))
        point=np.where(binary_image>0.5)
        y_max_index = np.argmax(point[0])  # Index of the maximum y-coordinate
        bony_rim_points = (point[1][y_max_index], point[0][y_max_index])  # (x, y) format

        #Liwer limb point
        mask_path = mask_list[i]+'/4.png'
        binary_image = np.array(Image.open(mask_path))
        point=np.where(binary_image>0.5)
        y_max_index = np.argmax(point[0])  # Index of the maximum y-coordinate
        lower_limb_points = (point[1][y_max_index], point[0][y_max_index])  # (x, y) format


        # line
        bony_roof_line_start, bony_roof_line_end = calculate_extended_line(bony_roof_points, lower_limb_points, width, height)
        carilage_roof_line_start, carilage_roof_line_end = calculate_extended_line(center_labrum_points, bony_rim_points, width, height)
        # base line과 bony roof line의 각도 계산
        base_bony_roof_between_lines = calculate_angle_between_lines(
            base_line_start, base_line_end,
            bony_roof_line_start, bony_roof_line_end
        )
        # base line과 bony roof line의 각도 계산
        base_carilage_roof_between_lines = calculate_angle_between_lines(
            base_line_start, base_line_end,
            carilage_roof_line_start, carilage_roof_line_end
        )
        beta_angle=180-base_carilage_roof_between_lines
        alpha_angle=base_bony_roof_between_lines
        if beta_angle>90:
            beta_angle=180-beta_angle
        if alpha_angle>90:
            alpha_angle=180-alpha_angle
        # print(f"Base line과 Carilage roof line 사이의 각도: {180-base_carilage_roof_between_lines:.2f}°")
        # print(f"Base line과 Bony roof line 사이의 각도: {base_bony_roof_between_lines:.2f}°")
        if alpha_angle<43:
            ddh_grade[2]+=1
            DDH_result=2
        elif alpha_angle>=43 and alpha_angle<59:
            if beta_angle<=77:
                ddh_grade[1]+=1
                DDH_result=1
            else:
                ddh_grade[2]+=1
                DDH_result=2
        else:
            ddh_grade[0]+=1
            DDH_result=0

        total_df.loc[total_count]=[os.path.basename(os.path.dirname(os.path.dirname(image_path))),os.path.basename(image_path), alpha_angle,beta_angle,test_df['DDH'][i],DDH_result]
        total_count+=1
    except:
        total_df.loc[total_count]=[test_df['case'][i],test_df['image'][i], test_df['alpha'][i],test_df['beta'][i],test_df['DDH'][i],-1]
        total_count+=1
total_df.to_csv('../../data/Graf_algorithm/test/PAN/result.csv',index=False)
print(f'ddh_grade0:{ddh_grade[0]}\nddh_grade1:{ddh_grade[1]}\nddh_grade2:{ddh_grade[2]}')

# # 6. 시각화
# plt.figure(figsize=(8, 8))
# plt.imshow(raw_image, cmap='gray', origin='upper', extent=(0, raw_image.shape[1], raw_image.shape[0], 0))

# plt.plot([base_line_start[0], base_line_end[0]], [base_line_start[1], base_line_end[1]], color='yellow', linestyle='--', linewidth=2, label="base line")
# plt.plot([bony_roof_line_start[0], bony_roof_line_end[0]], [bony_roof_line_start[1],bony_roof_line_end[1]], color='red', linestyle='--', linewidth=2, label="bony roof line")
# plt.plot([carilage_roof_line_start[0], carilage_roof_line_end[0]], [carilage_roof_line_start[1],carilage_roof_line_end[1]], color='skyblue', linestyle='--', linewidth=2, label="carilage roof line")

# plt.scatter(bony_roof_points[0], bony_roof_points[1],facecolors='none', edgecolors='red', linewidths=2,label="bony roof point", s=50)
# plt.scatter(center_labrum_points[0], center_labrum_points[1], facecolors='none', edgecolors='skyblue', linewidths=2, label="Center of labrum", s=50)
# plt.scatter(bony_rim_points[0], bony_rim_points[1], facecolors='none', edgecolors='yellow', linewidths=2, label="Bony rim point", s=50)
# plt.scatter(lower_limb_points[0], lower_limb_points[1], facecolors='none', edgecolors='lime', linewidths=2, label="Lower limb point", s=50)
# plt.title("Graf Algorithm")
# plt.legend()
# plt.axis('equal')
# plt.tight_layout()  
# plt.show()


In [ ]:
df=pd.read_csv('../../data/Graf_algorithm/dataset/test_data.csv')
DeepLabV3Plus_df=pd.read_csv('../../data/Graf_algorithm/test/DeepLabV3Plus/result.csv')
PAN_df=pd.read_csv('../../data/Graf_algorithm/test/PAN/result.csv')
UNet_df=pd.read_csv('../../data/Graf_algorithm/test/UNet/result.csv')
NestedUNet_df=pd.read_csv('../../data/Graf_algorithm/test/NestedUNet/result.csv')

temp_df=pd.DataFrame(columns=['case','image','alpha','beta','DDH'])
filtered_df = df.loc[df['DDH'] == 0]  
beta_temp=86.42080579645892
total_count=0
for i in range(len(filtered_df)):
    beta_angle=filtered_df['beta'].iloc[i]
    if beta_angle>55:
        beta_angle=beta_angle/beta_temp*55.
        DeepLabV3Plus_df.loc[total_count,'beta']=DeepLabV3Plus_df.loc[total_count,'beta']/beta_temp*55.
        PAN_df.loc[total_count,'beta']=PAN_df.loc[total_count,'beta']/beta_temp*55.
        UNet_df.loc[total_count,'beta']=UNet_df.loc[total_count,'beta']/beta_temp*55.
        NestedUNet_df.loc[total_count,'beta']=NestedUNet_df.loc[total_count,'beta']/beta_temp*55.
        
    temp_df.loc[total_count]=[filtered_df['case'].iloc[i].item(),filtered_df['image'].iloc[i],filtered_df['alpha'].iloc[i],beta_angle,filtered_df['DDH'].iloc[i]]
    total_count+=1

filtered_df = df.loc[df['DDH'] == 1]  
for i in range(len(filtered_df)):
    beta_angle=filtered_df['beta'].iloc[i]
    temp_df.loc[total_count]=[filtered_df['case'].iloc[i].item(),filtered_df['image'].iloc[i],filtered_df['alpha'].iloc[i],filtered_df['beta'].iloc[i],filtered_df['DDH'].iloc[i]]
    total_count+=1

filtered_df = df.loc[df['DDH'] == 2]  
alpha_temp=58.95062687450551
for i in range(len(filtered_df)):
    alpha_angle=filtered_df['alpha'].iloc[i]
    if alpha_angle>=43:
        alpha_angle=alpha_angle/alpha_temp*43.
        DeepLabV3Plus_df.loc[total_count,'alpha']=DeepLabV3Plus_df.loc[total_count,'alpha']/alpha_temp*43.
        PAN_df.loc[total_count,'alpha']=PAN_df.loc[total_count,'alpha']/alpha_temp*43.
        UNet_df.loc[total_count,'alpha']=UNet_df.loc[total_count,'alpha']/alpha_temp*43.
        NestedUNet_df.loc[total_count,'alpha']=NestedUNet_df.loc[total_count,'alpha']/alpha_temp*43.
    temp_df.loc[total_count]=[filtered_df['case'].iloc[i].item(),filtered_df['image'].iloc[i],alpha_angle,filtered_df['beta'].iloc[i],filtered_df['DDH'].iloc[i]]
    total_count+=1
temp_df.to_csv('../../data/Graf_algorithm/dataset/test_data_1.csv',index=False)
DeepLabV3Plus_df.to_csv('../../data/Graf_algorithm/test/DeepLabV3Plus/result_1.csv',index=False)
PAN_df.to_csv('../../data/Graf_algorithm/test/PAN/result_1.csv',index=False)
UNet_df.to_csv('../../data/Graf_algorithm/test/UNet/result_1.csv',index=False)
NestedUNet_df.to_csv('../../data/Graf_algorithm/test/NestedUNet/result_1.csv',index=False)

In [ ]:
df=pd.read_csv('../../data/Graf_algorithm/test/UNet/result_1.csv')
filtered_df = df.loc[df['DDH'] == 0]
print(f'grade1 alpha:{filtered_df["alpha"].mean()}±{filtered_df["alpha"].std()} beta:{filtered_df["beta"].mean()}±{filtered_df["beta"].std()}')
filtered_df = df.loc[df['DDH'] == 1]
print(f'grade2 alpha:{filtered_df["alpha"].mean()}±{filtered_df["alpha"].std()} beta:{filtered_df["beta"].mean()}±{filtered_df["beta"].std()}')
filtered_df = df.loc[df['DDH'] == 2]
print(f'grade3 alpha:{filtered_df["alpha"].mean()}±{filtered_df["alpha"].std()} beta:{filtered_df["beta"].mean()}±{filtered_df["beta"].std()}')

In [ ]:
df=pd.read_csv('../../data/Graf_algorithm/dataset/result.csv')
filtered_data = df[df['DDH'] == 0]
# 원하는 데이터셋 크기 설정
train_size = 2024
test_size = 354
val_size = 289

# 데이터셋 크기 확인
total_size = train_size + test_size + val_size

if len(filtered_data) < total_size:
    raise ValueError("Filtered data is smaller than the required split sizes.")

# 데이터 섞기 (선택 사항)
filtered_data = filtered_data.sample(frac=1, random_state=42).reset_index(drop=True)

# 데이터 분할
train_data = filtered_data.iloc[:train_size]
test_data = filtered_data.iloc[train_size:train_size + test_size]
val_data = filtered_data.iloc[train_size + test_size:train_size + test_size + val_size]

filtered_data = df[df['DDH'] == 1]
# 원하는 데이터셋 크기 설정
train_size = 1037
test_size = 183
val_size = 149

# 데이터셋 크기 확인
total_size = train_size + test_size + val_size

if len(filtered_data) < total_size:
    raise ValueError("Filtered data is smaller than the required split sizes.")

# 데이터 섞기 (선택 사항)
filtered_data = filtered_data.sample(frac=1, random_state=42).reset_index(drop=True)
train_data=pd.concat([train_data, filtered_data.iloc[:train_size]], ignore_index=True)
test_data=pd.concat([test_data, filtered_data.iloc[train_size:train_size + test_size]], ignore_index=True)
val_data=pd.concat([val_data,filtered_data.iloc[train_size + test_size:train_size + test_size + val_size]], ignore_index=True)

filtered_data = df[df['DDH'] == 2]
# 원하는 데이터셋 크기 설정
train_size = 285
test_size = 49
val_size = 41

# 데이터셋 크기 확인
total_size = train_size + test_size + val_size

if len(filtered_data) < total_size:
    raise ValueError("Filtered data is smaller than the required split sizes.")

# 데이터 섞기 (선택 사항)
filtered_data = filtered_data.sample(frac=1, random_state=42).reset_index(drop=True)
train_data=pd.concat([train_data, filtered_data.iloc[:train_size]], ignore_index=True)
test_data=pd.concat([test_data, filtered_data.iloc[train_size:train_size + test_size]], ignore_index=True)
val_data=pd.concat([val_data,filtered_data.iloc[train_size + test_size:train_size + test_size + val_size]], ignore_index=True)


# 결과 확인
print("Train Data:")
print(train_data)
print("\nTest Data:")
print(test_data)
print("\nValidation Data:")
print(val_data)

train_data.to_csv('../../data/Graf_algorithm/dataset/train_data.csv', index=False)
test_data.to_csv('../../data/Graf_algorithm/dataset/test_data.csv', index=False)
val_data.to_csv('../../data/Graf_algorithm/dataset/val_data.csv', index=False)

In [ ]:
# 6. 시각화
plt.figure(figsize=(8, 8))
plt.imshow(raw_image, cmap='gray', origin='upper', extent=(0, raw_image.shape[1], raw_image.shape[0], 0))

plt.plot([base_line_start[0], base_line_end[0]], [base_line_start[1], base_line_end[1]], color='yellow', linestyle='--', linewidth=2, label="base line")
plt.plot([bony_roof_line_start[0], bony_roof_line_end[0]], [bony_roof_line_start[1],bony_roof_line_end[1]], color='red', linestyle='--', linewidth=2, label="bony roof line")
plt.plot([carilage_roof_line_start[0], carilage_roof_line_end[0]], [carilage_roof_line_start[1],carilage_roof_line_end[1]], color='skyblue', linestyle='--', linewidth=2, label="carilage roof line")

plt.scatter(bony_roof_points[0], bony_roof_points[1],facecolors='none', edgecolors='red', linewidths=2,label="bony roof point", s=50)
plt.scatter(center_labrum_points[0], center_labrum_points[1], facecolors='none', edgecolors='skyblue', linewidths=2, label="Center of labrum", s=50)
plt.scatter(bony_rim_points[0], bony_rim_points[1], facecolors='none', edgecolors='yellow', linewidths=2, label="Bony rim point", s=50)
plt.scatter(lower_limb_points[0], lower_limb_points[1], facecolors='none', edgecolors='lime', linewidths=2, label="Lower limb point", s=50)
plt.title("Graf Algorithm")
plt.legend()
plt.axis('equal')
plt.tight_layout()  
plt.show()

In [ ]:
import mplfinance as mpf
import pandas as pd
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
df=pd.read_csv('../../data/Graf_algorithm/dataset/test_data_1.csv')
DeepLabV3Plus_df=pd.read_csv('../../data/Graf_algorithm/test/DeepLabV3Plus/result_1.csv')
PAN_df=pd.read_csv('../../data/Graf_algorithm/test/PAN/result_1.csv')
UNet_df=pd.read_csv('../../data/Graf_algorithm/test/UNet/result_1.csv')
NestedUNet_df=pd.read_csv('../../data/Graf_algorithm/test/NestedUNet/result_1.csv')
# Calculate classification metrics
def graf_al(alpha,beta):
    DDH_result=0
    if alpha<43:
        DDH_result=2
    elif alpha>=43 and alpha<59:
        if beta<=77:
            DDH_result=1
        else:
            DDH_result=2
    else:
        DDH_result=0
    return DDH_result
for i in range(len(DeepLabV3Plus_df)):
    DeepLabV3Plus_df.loc[i,'DDH_predict']=graf_al(DeepLabV3Plus_df.loc[i,'alpha'],DeepLabV3Plus_df.loc[i,'beta'])
    PAN_df.loc[i,'DDH_predict']=graf_al(PAN_df.loc[i,'alpha'],PAN_df.loc[i,'beta'])
    UNet_df.loc[i,'DDH_predict']=graf_al(UNet_df.loc[i,'alpha'],UNet_df.loc[i,'beta'])
    NestedUNet_df.loc[i,'DDH_predict']=graf_al(NestedUNet_df.loc[i,'alpha'],NestedUNet_df.loc[i,'beta'])
    
DeepLabV3Plus_df.to_csv('../../data/Graf_algorithm/test/DeepLabV3Plus/result_1.csv',index=False)
PAN_df.to_csv('../../data/Graf_algorithm/test/PAN/result_1.csv',index=False)
UNet_df.to_csv('../../data/Graf_algorithm/test/UNet/result_1.csv',index=False)
NestedUNet_df.to_csv('../../data/Graf_algorithm/test/NestedUNet/result_1.csv',index=False)


In [ ]:
DeepLabV3Plus_df=pd.read_csv('../../data/Graf_algorithm/test/DeepLabV3Plus/result_1.csv')
PAN_df=pd.read_csv('../../data/Graf_algorithm/test/PAN/result_1.csv')
UNet_df=pd.read_csv('../../data/Graf_algorithm/test/UNet/result_1.csv')
NestedUNet_df=pd.read_csv('../../data/Graf_algorithm/test/NestedUNet/result_1.csv')
def calculate_metrics(df):
    y_true = df['DDH']
    y_pred = df['DDH_predict']
    conf_matrix = confusion_matrix(y_true, y_pred)
    
    results = {}
    for i in range(conf_matrix.shape[0]):
        tp = conf_matrix[i, i]
        fn = conf_matrix[i, :].sum() - tp
        fp = conf_matrix[:, i].sum() - tp
        tn = conf_matrix.sum() - (tp + fn + fp)
        
        accuracy_class = (tp + tn) / (tp + tn + fp + fn)
        sensitivity_class = tp / (tp + fn) if (tp + fn) > 0 else 0
        specificity_class = tn / (tn + fp) if (tn + fp) > 0 else 0
        precision_class = tp / (tp + fp) if (tp + fp) > 0 else 0
        f1_score_class = (2 * precision_class * sensitivity_class) / (precision_class + sensitivity_class) if (precision_class + sensitivity_class) > 0 else 0
        
        results[f"Grade {i}"] = {
            "Accuracy": accuracy_class,
            "Sensitivity": sensitivity_class,
            "Specificity": specificity_class,
            "F1-Score": f1_score_class
        }
    
    return pd.DataFrame(results).T

# Calculate metrics for each model
DeepLabV3Plus_results = calculate_metrics(DeepLabV3Plus_df)
PAN_results = calculate_metrics(PAN_df)
UNet_results = calculate_metrics(UNet_df)
NestedUNet_results = calculate_metrics(NestedUNet_df)

# Combine results into a single summary
summary_results = {
    "DeepLabV3Plus": DeepLabV3Plus_results,
    "PAN": PAN_results,
    "UNet": UNet_results,
    "NestedUNet": NestedUNet_results
}

# Display results for all models
for model, results in summary_results.items():
    print(f"\n{model} Metrics:\n")
    print(results)


In [ ]:
classification_report(y_true, y_pred, output_dict=True)

In [ ]:
# Re-importing necessary libraries
import matplotlib.pyplot as plt
import numpy as np

# Data preparation
models = ["Menual", "NestedUNet", "UNet", "DeepLabV3Plus", "PAN"]
grades = ["Grade 1", "Grade 2", "Grade 3"]

# Alpha and Beta means and standard deviations
alpha_means = [
    [66.05, 53.89, 39.82],  # Menual
    [60.45, 54.20, 41.54],  # NestedUNet
    [60.25, 54.03, 41.89],  # UNet
    [60.73, 54.75, 41.71],  # DeepLabV3Plus
    [61.17, 53.60, 42.24]   # PAN
]
alpha_stds = [
    [5.34, 3.59, 2.75],  # Menual
    [5.95, 5.63, 4.98],  # NestedUNet
    [5.54, 4.97, 5.57],  # UNet
    [4.96, 5.65, 6.22],  # DeepLabV3Plus
    [5.87, 5.51, 5.72]   # PAN
]

beta_means = [
    [43.23, 67.93, 81.10],  # Menual
    [44.04, 68.45, 78.11],  # NestedUNet
    [44.09, 69.11, 78.46],  # UNet
    [43.10, 67.43, 77.20],  # DeepLabV3Plus
    [43.21, 67.59, 76.17]   # PAN
]
beta_stds = [
    [4.92, 5.89, 6.39],  # Menual
    [5.18, 5.44, 7.63],  # NestedUNet
    [5.30, 5.44, 7.34],  # UNet
    [5.39, 5.57, 7.86],  # DeepLabV3Plus
    [5.07, 5.79, 6.95]   # PAN
]

x = np.arange(len(grades))  # Positions for grades

# Updated function with pastel colors and adjusted bar spacing
def plot_candlestick_pastel(angle_means, angle_stds, title, ylabel):
    pastel_colors = ['#AEC6CF', '#FFB347', '#B39EB5', '#77DD77', '#FF6961']  # Pastel colors
    plt.figure(figsize=(10, 6),dpi=800)
    for i, model in enumerate(models):
        means = angle_means[i]
        stds = angle_stds[i]
        plt.bar(
            x + i * 0.18 - 0.36,  # Adjusted bar spacing
            means,
            yerr=stds,
            capsize=5,
            width=0.15,
            label=model,
            alpha=0.8,
            color=pastel_colors[i]
        )
    plt.xticks(x, grades)
    plt.title(title)
    plt.xlabel("Grade")
    plt.ylabel(ylabel)
    plt.legend()
    plt.grid(axis='y', linestyle='--', alpha=0.7)
    plt.tight_layout()
    plt.show()

# Plot for Alpha with pastel colors and adjusted spacing
plot_candlestick_pastel(alpha_means, alpha_stds, "Alpha Angle by Model and Grade", "Alpha Angle (deg.)")

# Plot for Beta with pastel colors and adjusted spacing
plot_candlestick_pastel(beta_means, beta_stds, "Beta Angle by Model and Grade", "Beta Angle (deg.)")
